# Deep Learning - Proyecto Final

## Predicción de Tiempo de recorrido en actividades de Trail Running

## Procesamiento de Datos

La lógica de procesamiento de datos se encuentra implementada en la clase `DataProcessor` dentro del archivo [data_processor.py](utils/data_processor.py). El dispositivo de registro es un reloj Polar Vantage V. Los datos de Polar se encuentran estructurados en formato JSON por lo que es importante poder pasarlos a un formato tabular antes de su procesamiento en el modelo.

El procesamiento de estos datos consiste en extraer y transformar información de archivos JSON que contienen registros de actividades de trail running. El script filtra las actividades por tipo de deporte (TRAIL_RUNNING) y duración (entre 4 y 6 horas). Para cada actividad válida, se extraen muestras de sensores como frecuencia cardíaca, altitud, distancia, temperatura, cadencia y velocidad.

Las muestras se organizan en un DataFrame, donde se realizan varias tareas de ingeniería de características:

Distancia: Se calcula la diferencia entre muestras consecutivas para obtener el avance por intervalo.
Altitud: Se suavizan los datos y se calculan diferencias de elevación, ganancia y pérdida acumulada.
Duración: Se calcula el tiempo acumulado entre muestras.

El procesamiento se realiza de forma eficiente, permitiendo el uso de múltiples hilos para acelerar la conversión de archivos. Los datos procesados se guardan en archivos CSV estructurados, listos para su análisis y modelado.

## Modelo

## Entrenamiento

## Evaluación

## Discusión y Conclusiones